# AVIATION SAFETY NETWORK - DATA TRANSFORM - V3

En este notebook realizaremos una ultima transformacion sobre el dataset resultante del EDA realizado.

En particular: 
1. Imputaremos los datos para asi hacer uso en el dashboard:
    - A los datos numericos, los imputamos por la mediana, puesto que la distribucion es asimetrica.
    - A los datos categoricos los imputamos segun la distribucion.
2. Obtenemos latitude y longitude para cada pais y continente usando la libreria geopy.


In [1]:
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="proyecto")

In [2]:
data = pd.read_csv('../../data/transform/asn_final_eda.csv')

In [3]:
data.shape

(19253, 40)

In [4]:
# Numero de nulos totales

data.isnull().sum().sum()

118128

In [5]:
# Seleccionamos las columnas categoricas y las columnas numericas.

cols_categoricas = data.select_dtypes(exclude=np.number).columns
cols_numericas = data.select_dtypes(include=np.number).columns

In [6]:
def imputar_distribucion(dataframe, columnas):
    # Calcula la distribucion de valores y elige aleatoriamente un valor para las variables categorias en base a ello.
    # Esto se realiza para elemento en columnas.
    for c in columnas:
        valores_faltantes = dataframe[c].isnull().sum()
        categorias = dataframe[c].value_counts().index
        distribucion = dataframe[c].value_counts(normalize=True)
        
        # Generar valores imputados
        imputados = np.random.choice(categorias, size=valores_faltantes, p=distribucion)
        
        # Reemplazar los valores faltantes con los imputados
        dataframe.loc[dataframe[c].isnull(), c] = imputados
    
    return dataframe

def imputar_mediana(dataframe, columnas):
    for c in columnas:
        mediana = dataframe[c].median()
        
        # Reemplazar los valores faltantes con la mediana
        dataframe[c].fillna(mediana, inplace=True)
    
    return dataframe

In [7]:
new_df = data.copy()

new_df = imputar_distribucion(new_df, cols_categoricas)
new_df = imputar_mediana(new_df, cols_numericas)

In [8]:
new_df.shape

(19253, 40)

In [9]:
new_df.isnull().sum().sum()

0

In [10]:
# Eliminamos del dataset todos aquellos datos con el continente unknown

indice = False == (new_df.continente == 'unknown')
new_df = new_df.loc[indice]

In [28]:
# Esta funcion hace uso de geopy.
# Dado un pais/continente devuelve la latitud y la longitud. Estos valores son usados posteriormente para graficos de mapa.

def get_coords(sitios):
    output = {}
    for v in sitios:
        try:
            location = geolocator.geocode(v)
            if location is not None:
                lat = location.latitude
                long = location.longitude
            else:
                lat = long = np.nan
        except:
            lat = long = np.nan

        output[v] = [lat, long]
        
    return output

# Dado un pais/continente devuelve su latitud si el parametro es True o longitud si es False
def extract_coords(pais, latitude=True):
    if latitude:
        return map[pais][0]
    return map[pais][1]

In [29]:
print(new_df.pais_accidente.unique().size)

# Obtenemos las coordenadas geograficas.
map = get_coords(new_df.pais_accidente.unique().tolist())

205


In [41]:
latitude = new_df.pais_accidente.apply(lambda v: extract_coords(v, True))
longitude = new_df.pais_accidente.apply(lambda v: extract_coords(v, False))

new_df['lat'] = latitude
new_df['lon'] = longitude

In [48]:
print(new_df.continente.unique().size)
map = get_coords(new_df.continente.unique().tolist())

7


In [50]:
latitude = new_df.continente.apply(lambda v: extract_coords(v, True))
longitude = new_df.continente.apply(lambda v: extract_coords(v, False))

new_df['lat_cont'] = latitude
new_df['lon_cont'] = longitude

In [51]:
new_df.shape

(18162, 45)

In [52]:
# Almacenamos el dataset que luego sera usado por streamlit
new_df.to_csv('../../data/transform/asn_final_v3.csv', header=True, index=False, mode='w')